# Answering ex. Business Questions using SQL

In this project we'll be answering example business questions using a database that stores data for a music store called chinook. The image below shows the schema (visual relationships between tables in the database).

### Schema

![image](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

We'll start by loading our data and quickly crosschecking the contents against the schema.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Looks like we're set. Let's get to business!

## Problem 1

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name | Genre|
| --- | --- |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

You'll need to write a query to find out which genres sell the most tracks in the USA, write up a summary of your findings, and make a recommendation for the three artists whose albums we should purchase for the store.



I'll start by examing the tables that will be needed for this analysis.

- genre
- track
- invoice
- invoice_line

In [3]:
%%sql
SELECT * FROM genre LIMIT 3;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal


In [4]:
%%sql
SELECT * FROM track LIMIT 3;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99


In [5]:
%%sql
SELECT * FROM invoice LIMIT 3;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98


In [6]:
%%sql
SELECT * FROM invoice_line LIMIT 3;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1


Now, I need to join these table to come up with the quantity sold for each genre. I'll also calculate the percentage of total using a subquery that gives me the total for total tracks sold in the USA.

In [7]:
%%sql

WITH total_tracks AS
    (
    SELECT
        i.billing_country country,
        SUM(il.quantity) us_quantity
    FROM invoice_line il
    LEFT JOIN invoice i ON i.invoice_id = il.invoice_id
    WHERE i.billing_country = 'USA'
    ),
     quantity_table AS
    (
    SELECT 
        i.billing_country country,
        g.name genre,
        SUM(il.quantity) quantity
    FROM invoice i
    LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
    LEFT JOIN track t ON t.track_id = il.track_id
    LEFT JOIN genre g ON g.genre_id = t.genre_id
    WHERE i.billing_country = 'USA'
    GROUP BY genre
    ORDER BY 2 DESC
    )

SELECT
    qt.genre,
    qt.quantity,
    ROUND(CAST(qt.quantity AS float) / 
          CAST(tt.us_quantity AS float)* 100,0) percent_of_total
FROM quantity_table qt
LEFT JOIN total_tracks tt ON tt.country = qt.country
ORDER BY 3 DESC
LIMIT 10;

Done.


genre,quantity,percent_of_total
Rock,561,53.0
Metal,124,12.0
Alternative & Punk,130,12.0
R&B/Soul,53,5.0
Blues,36,3.0
Alternative,35,3.0
Pop,22,2.0
Latin,22,2.0
Hip Hop/Rap,20,2.0
Reggae,6,1.0


## Problem 1: Conclusion

From historical sales data, we generated a table that shows us track sales based on genres. Of the four albums, Regal will most likely sell less than the other 3. We should purchase:

- Red Tone
- Meteor and the Girls
- Slim Jim Bites.

## Problem 2

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.


In [8]:
%%sql
SELECT
    e1.first_name||' '||e1.last_name sales_rep,
    e2.first_name||' '||e2.last_name manager,
    e1.hire_date hire_date,
    SUM(i.total) total_sales,
    COUNT(c.customer_id) total_customers,
    SUM(i.total) / COUNT(c.customer_id) avg_sales_per_customer
FROM employee e1
LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
LEFT JOIN customer c ON e1.employee_id = c.support_rep_id
LEFT JOIN invoice i ON i.customer_id = c.customer_id
WHERE e1.title = 'Sales Support Agent'
GROUP BY sales_rep
ORDER BY total_sales DESC;


Done.


sales_rep,manager,hire_date,total_sales,total_customers,avg_sales_per_customer
Jane Peacock,Nancy Edwards,2017-04-01 00:00:00,1731.510000000004,212,8.16750000000002
Margaret Park,Nancy Edwards,2017-05-03 00:00:00,1584.0000000000032,214,7.40186915887852
Steve Johnson,Nancy Edwards,2017-10-17 00:00:00,1393.9200000000028,188,7.414468085106398


From the table above, Jane performs better than Margaret and Steve in terms of sales. We can also see that Jane's average sales per customer is higher than the others.

In [9]:
%%sql
SELECT * FROM employee WHERE title = 'Sales Support Agent';

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


## Problem 2: Conclusion

After examing the employee details, we can see that all the sales support agents report to the same manager and are from the same area. Jane has seniority when it comes to hiring, however, she only started one month before Margaret and six months before Steve. The last notable detail is that she is the youngest of the 3. She was born in 1973 while Margaret and Steve were born in 1947 and 1965, respectively. Perhaps a combination of starting at the company sooner and her younger age contributed to her ability to sell more (there's a possibility of age discrimination of we enforce policies based off of age).

## Problem 3

Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis.


In [10]:
%%sql
WITH country_customers AS
    (
    SELECT
        c.country country,
        COUNT(c.customer_id) n_customers
    FROM customer c
    GROUP BY 1
    ORDER BY 2 DESC
    ), 
    country_sales AS
    (
    SELECT
        c.country country,
        SUM(i.total) n_sales,
        COUNT(i.invoice_id) n_orders
    FROM customer c
    LEFT JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY 1
    ORDER BY 2 DESC
    ),
    country_sales_avg AS
    (
    SELECT
        CASE
            WHEN cc.n_customers = 1 THEN 'Other'
            ELSE cc.country
            END
            AS sales_country,
        SUM(cc.n_customers) n_customers,
        SUM(cs.n_sales) n_sales,
        SUM(cs.n_sales) / SUM(cc.n_customers) avg_sales,
        SUM(cs.n_sales) / SUM(cs.n_orders) avg_order        
    FROM country_customers cc
    LEFT JOIN country_sales cs ON cc.country = cs.country
    GROUP BY 1
    ORDER BY 2 DESC
    ),
    country_sales_avg_sort AS
    (
    SELECT
        csa.*,
        CASE
            WHEN sales_country = 'Other' THEN 1
            ELSE 0
            END 
            AS sort
    FROM country_sales_avg csa
    ORDER BY sort ASC
    )

SELECT
    sales_country country,
    n_customers customers,
    n_sales total_sales,
    avg_sales customer_lifetime_value,
    avg_order average_order
FROM country_sales_avg_sort;

Done.


country,customers,total_sales,customer_lifetime_value,average_order
USA,13,1040.4900000000005,80.03769230769234,7.942671755725194
Canada,8,535.59,66.94875,7.047236842105264
Brazil,5,427.6799999999999,85.53599999999997,7.011147540983605
France,5,389.07000000000005,77.81400000000001,7.781400000000001
Germany,4,334.61999999999995,83.65499999999999,8.161463414634145
United Kingdom,3,245.52,81.84,8.768571428571429
Czech Republic,2,273.23999999999995,136.61999999999998,9.107999999999999
India,2,183.14999999999995,91.57499999999997,8.72142857142857
Portugal,2,185.13,92.565,6.3837931034482756
Other,15,1094.9399999999998,72.996,7.4485714285714275


## Problem 3: Conclusion
From our sales data there are a few countries that have data that stands out:

- United Kingdom
- Czech Republic
- India

The average order size from those three countries are range from 8.7-9.1 while the rest range between 6.4-9.2. Czech Republic also has a significantly higher Lifetime Customer Value (LCV) than the rest coming in $47 higher than the country with the second highest LCV.

## Problem 4

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent you from getting a 100% accurate answer to your question. In this instance, we have two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.


In [34]:
%%sql
WITH list_invoice_songs AS
        (
        SELECT
            i.invoice_id invoice_id,
            MIN(il.track_id) one_track
        FROM invoice i
        INNER JOIN invoice_line il on i.invoice_id = il.invoice_id
        GROUP BY 1
        ),
    list_invoice_albums AS
        (
        SELECT
             lis.*,
             t.album_id
        FROM list_invoice_songs lis
        LEFT JOIN track t ON t.track_id = lis.one_track
        )
SELECT
    album_purchase,
    COUNT(*) invoice_count,
    CAST(COUNT(*) AS float) / (SELECT COUNT(*)FROM invoice) percentage
FROM
    (
    SELECT
        lis.*,
        CASE
            WHEN
                (
                SELECT
                    il.track_id
                FROM invoice_line il
                WHERE il.invoice_id = lis.invoice_id


                EXCEPT

                SELECT
                     t.track_id
                 FROM track t
                 WHERE t.album_id = lis.album_id
                ) IS NULL
            AND
                (
                SELECT
                     t.track_id
                FROM track t
                WHERE t.album_id = lis.album_id


                EXCEPT


                SELECT
                    il.track_id
                FROM invoice_line il
                WHERE il.invoice_id = lis.invoice_id
                ) IS NULL
            THEN 'YES'
            ELSE 'NO'
        END AS album_purchase
    FROM list_invoice_albums lis
    )
GROUP BY album_purchase;

Done.


album_purchase,invoice_count,percentage
NO,500,0.8143322475570033
YES,114,0.18566775244299674


## Problem 4: Conclusion
Album purchase make up almost 20% of the purchases. I recommend against